Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST_sanitized.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (8971, 28, 28) (8971,)
Test set (8676, 28, 28) (8676,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (8971, 28, 28, 1) (8971, 10)
Test set (8676, 28, 28, 1) (8676, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [19]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        print_shape(data)
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        print_shape(conv)
        hidden = tf.nn.relu(conv + layer1_biases)
        print_shape(hidden)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        print_shape(conv)
        hidden = tf.nn.relu(conv + layer2_biases)
        print_shape(hidden)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        print_shape(reshape)
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        print_shape(hidden)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

[16, 28, 28, 1]
[16, 14, 14, 16]
[16, 14, 14, 16]
[16, 7, 7, 16]
[16, 7, 7, 16]
[16, 784]
[16, 64]
[8971, 28, 28, 1]
[8971, 14, 14, 16]
[8971, 14, 14, 16]
[8971, 7, 7, 16]
[8971, 7, 7, 16]
[8971, 784]
[8971, 64]
[8676, 28, 28, 1]
[8676, 14, 14, 16]
[8676, 14, 14, 16]
[8676, 7, 7, 16]
[8676, 7, 7, 16]
[8676, 784]
[8676, 64]


In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('-----')
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
-----
Minibatch loss at step 0: 2.693478
Minibatch accuracy: 12.5%
Validation accuracy: 8.3%
-----
Minibatch loss at step 50: 1.922897
Minibatch accuracy: 37.5%
Validation accuracy: 49.7%
-----
Minibatch loss at step 100: 1.114455
Minibatch accuracy: 68.8%
Validation accuracy: 72.3%
-----
Minibatch loss at step 150: 0.583641
Minibatch accuracy: 81.2%
Validation accuracy: 74.0%
-----
Minibatch loss at step 200: 0.982278
Minibatch accuracy: 75.0%
Validation accuracy: 76.9%
-----
Minibatch loss at step 250: 1.129542
Minibatch accuracy: 68.8%
Validation accuracy: 77.6%
-----
Minibatch loss at step 300: 1.087130
Minibatch accuracy: 75.0%
Validation accuracy: 77.6%
-----
Minibatch loss at step 350: 0.801023
Minibatch accuracy: 81.2%
Validation accuracy: 77.5%
-----
Minibatch loss at step 400: 0.950544
Minibatch accuracy: 62.5%
Validation accuracy: 78.6%
-----
Minibatch loss at step 450: 0.805151
Minibatch accuracy: 75.0%
Validation accuracy: 79.1%
-----
Minibatch loss at step 500

### strides = 1

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size * image_size * depth // 4, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        print_shape(data)
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        print_shape(conv)
        hidden = tf.nn.relu(conv + layer1_biases)
        print_shape(hidden)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        print_shape(conv)
        hidden = tf.nn.relu(conv + layer2_biases)
        print_shape(hidden)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        print_shape(reshape)
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        print_shape(hidden)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('-----')
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
-----
Minibatch loss at step 0: 5.155722
Minibatch accuracy: 18.8%
Validation accuracy: 11.9%
-----
Minibatch loss at step 50: 2.224962
Minibatch accuracy: 18.8%
Validation accuracy: 36.1%
-----
Minibatch loss at step 100: 0.973519
Minibatch accuracy: 75.0%
Validation accuracy: 64.6%
-----
Minibatch loss at step 150: 0.726416
Minibatch accuracy: 68.8%
Validation accuracy: 72.1%
-----
Minibatch loss at step 200: 1.170912
Minibatch accuracy: 62.5%
Validation accuracy: 76.5%
-----
Minibatch loss at step 250: 0.999724
Minibatch accuracy: 68.8%
Validation accuracy: 75.8%
-----
Minibatch loss at step 300: 0.970350
Minibatch accuracy: 62.5%
Validation accuracy: 78.1%
-----
Minibatch loss at step 350: 0.750465
Minibatch accuracy: 68.8%
Validation accuracy: 77.9%
-----
Minibatch loss at step 400: 0.953787
Minibatch accuracy: 75.0%
Validation accuracy: 78.2%
-----
Minibatch loss at step 450: 0.688610
Minibatch accuracy: 81.2%
Validation accuracy: 79.4%
-----
Minibatch loss at step 50

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [18]:
def print_shape(data):
    print(data.get_shape().as_list())

In [25]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        max_pooled = tf.nn.max_pool(hidden, 
                                    ksize=[1, 2, 2, 1], 
                                    strides=[1, 2, 2, 1], 
                                    padding='SAME')
        
        conv = tf.nn.conv2d(max_pooled, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        max_pooled = tf.nn.max_pool(hidden, 
                                    ksize=[1, 2, 2, 1], 
                                    strides=[1, 2, 2, 1], 
                                    padding='SAME')
        
        shape = max_pooled.get_shape().as_list()
        reshape = tf.reshape(max_pooled, [shape[0], shape[1] * shape[2] * shape[3]])

        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

[16, 28, 28, 16]
[16, 14, 14, 16]
[16, 14, 14, 16]
[16, 7, 7, 16]
[16, 784]
[8971, 28, 28, 16]
[8971, 14, 14, 16]
[8971, 14, 14, 16]
[8971, 7, 7, 16]
[8971, 784]
[8676, 28, 28, 16]
[8676, 14, 14, 16]
[8676, 14, 14, 16]
[8676, 7, 7, 16]
[8676, 784]


In [26]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('-----')
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
-----
Minibatch loss at step 0: 3.764734
Minibatch accuracy: 18.8%
Validation accuracy: 10.2%
-----
Minibatch loss at step 50: 2.196135
Minibatch accuracy: 31.2%
Validation accuracy: 47.2%
-----
Minibatch loss at step 100: 1.053318
Minibatch accuracy: 75.0%
Validation accuracy: 70.1%
-----
Minibatch loss at step 150: 0.496711
Minibatch accuracy: 81.2%
Validation accuracy: 72.4%
-----
Minibatch loss at step 200: 1.105381
Minibatch accuracy: 56.2%
Validation accuracy: 76.6%
-----
Minibatch loss at step 250: 0.937704
Minibatch accuracy: 68.8%
Validation accuracy: 78.4%
-----
Minibatch loss at step 300: 1.082344
Minibatch accuracy: 75.0%
Validation accuracy: 79.0%
-----
Minibatch loss at step 350: 0.732909
Minibatch accuracy: 75.0%
Validation accuracy: 77.0%
-----
Minibatch loss at step 400: 1.122758
Minibatch accuracy: 56.2%
Validation accuracy: 79.4%
-----
Minibatch loss at step 450: 0.537831
Minibatch accuracy: 87.5%
Validation accuracy: 80.6%
-----
Minibatch loss at step 50

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---